# Import necessary libraries

In [2]:
import os
import pandas as pd
import numpy as np
import cv2
import tkinter as tk
import tkinter.font as font
from datetime import datetime

# Preprocessing images

In [4]:
def preprocess_image(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    eq = cv2.equalizeHist(blurred)
    return eq

# Training

In [5]:
def train():
	print("training part initiated !")

	recog = cv2.face.LBPHFaceRecognizer_create()

	dataset = 'sizeResized600'

	paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]

	faces = []
	ids = []
	  
	for path in paths:
		# Assuming the filename format is "name-id-count-etc.jpg"
		parts = path.split(os.sep)[-1].split('_')  # os.sep is platform-independent
		label = parts[1]  # ID is now in the second position
		ids.append(int(label))
		faces.append(cv2.imread(path, 0))


	recog.train(faces, np.array(ids))
	recog.save('model600.yml')
 
	#creating excel file to store names and ids
	paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]
	names = []
	ids = []
	for path in paths:
		# Assuming the filename format is "name-id-count-etc.jpg"
		parts = path.split(os.sep)[-1].split('_')
		ids.append(int(parts[1]))  # ID is now in the second position
		names.append(parts[0])  # Name is still in the first position


	# print(names,ids,sep="\n\n")
	data = {'Name': names, 'ID': ids}
	df = pd.DataFrame(data)
	df.drop_duplicates(keep='last',inplace=True)
	df = df.sort_values(by='ID')

	# Save the DataFrame to an Excel file
	df.to_excel('names_ids.xlsx', index=False)


	#appending new student data to students excel file
	df1 = pd.read_excel("names_ids.xlsx")
	df2 = pd.read_excel("students.xlsx")
	missing_ids = df1['ID'].loc[~df1['ID'].isin(df2['ID'])].tolist()
	df2 = pd.concat([df2, df1.loc[df1['ID'].isin(missing_ids)]], ignore_index=True)
	df2 = df2.sort_values(by="ID")
	df2.to_excel("students.xlsx", index=False)
	print("training part ended !")
	return

# collect input using GUI

In [6]:
def register():
    """
    Collects user input for name and ID through a Tkinter GUI,
    processes and stores captured face images using OpenCV.

    Displays a separate window for face detection and image capture.
    """
    def get_info_and_destroy():
        """Retrieves user input from the Entry widgets and stores them in variables."""
        name = name_entry.get()
        id = id_entry.get()

        # Validate or process name and ID here if needed
        if not name or not id:
            # Display an error message
            print("Enter valid name and id")
            return
        
        # Capture face images
        capture_faces(name, id)

        root.destroy()  # Destroy the input window

    # Create the main window for name and ID collection
    root = tk.Tk()
    root.title("Student Registration")

    # Set window geometry and background color
    root.geometry("480x240")  # Adjust width and height as desired
    root.config(bg='#607B9E')

    # Create a frame to hold labels and entries, centered in the window
    frame = tk.Frame(root, bg="#333333")
    frame.pack(expand=True)

    # Create labels for name and ID, placed on the left side of the frame
    name_label = tk.Label(frame, text="Name:", width=6, height=1, bg="#e8de1a")
    name_label.grid(row=0, column=0, sticky="w", padx=(10,5), pady=(10,5))

    id_label = tk.Label(frame, text="ID:",width=6, height=1, bg="#e8de1a",)
    id_label.grid(row=1, column=0, sticky="w", padx=(10,5), pady=(5,5))

    # Create Entry widgets for user input, placed on the right side of the frame
    name_entry = tk.Entry(frame, bg="#e8de1a")
    name_entry.grid(row=0, column=1, padx=(5,10), pady=(10,5))  # Add padding between label and entry

    id_entry = tk.Entry(frame, bg="#e8de1a")
    id_entry.grid(row=1, column=1, padx=(5,10), pady=(5,5))  # Add padding between label and entry

    # Create a button to trigger get_info_and_destroy, placed below entries in the frame
    submit_button = tk.Button(frame, text="Submit", command=get_info_and_destroy, bg="#e8de1a", height=1)
    submit_button.grid(row=2, columnspan=2, pady=5)  # Span across both columns and add padding

    root.mainloop()  # Start the event loop for the input window

def capture_faces(name, id):
    """Captures face images using OpenCV."""

    count = 1

    cap = cv2.VideoCapture(0)

    filename = "haarcascade_frontalface_default.xml"

    cascade = cv2.CascadeClassifier(filename)

    while True:
        _, frm = cap.read()

        # gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)

        gray = preprocess_image(frm)

        faces = cascade.detectMultiScale(gray, 1.4, 1)

        for x,y,w,h in faces:
            cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (160, 160))

            cv2.imwrite(f"sizeResized600/{name}_{id}_{count}.jpg", roi)
            count = count + 1
            cv2.putText(frm, f"{count}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
            cv2.imshow("new", roi)


        cv2.imshow("identify", frm)

        if cv2.waitKey(1) == 27 or count > 600:
            cv2.destroyAllWindows()
            cap.release()
            train()
            break

# attendance marking implementation

In [7]:
def mark_attendance4():                  
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        att_file = 'students.xlsx'
        df = pd.read_excel(att_file)
        
        preds = []
        
        filename = "haarcascade_frontalface_default.xml"

        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()
            
            gray = preprocess_image(frm)
            faces = cascade.detectMultiScale(gray, 1.3, 2)

            for x,y,w,h in faces:
                cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                cv2.putText(frm, f"{len(preds)}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                    preds.append(label[0])
                    if  len(preds) >= 20:
                        pred_dict = {f"{pred}":preds.count(pred) for pred in set(preds)}
                        max = 0
                        crct_pred = None
                        for  k,v in pred_dict.items():
                            if max < v:
                                max = v
                                crct_pred = k
                        print(preds,crct_pred,sep="\n")
                        
                        # Append the attendance record to the Excel file
                        date_today = datetime.now().strftime("%Y-%m-%d")
                        if date_today not in df.columns:
                            df[date_today] = np.nan
                        
                        if pd.isna(df.loc[df['ID'] == label[0], date_today].iloc[0]):
                            df.loc[df['ID'] == label[0], date_today] = datetime.now().strftime("%H:%M:%S")
                        else:
                            df.loc[df['ID'] == label[0], date_today] = df.loc[df['ID'] == label[0], date_today].astype(str) + ',  ' + datetime.now().strftime("%H:%M:%S")

                        preds.clear()

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                # Save the updated DataFrame back to the Excel file
                df.to_excel(att_file,index=False)
                cv2.destroyAllWindows()
                cap.release()
                break
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)

    recog = cv2.face.LBPHFaceRecognizer_create()

    recog.read('model600.yml')
    real_time_recognition()

In [9]:
def real_time_recognition_test():
	cap = cv2.VideoCapture(0)
	# cap = cv2.VideoCapture(f'http://192.168.202.85:8080/video')

	filename = "haarcascade_frontalface_default.xml"


	cascade = cv2.CascadeClassifier(filename)
	while True:
		_, frm = cap.read()

		gray = preprocess_image(frm)

		faces = cascade.detectMultiScale(gray, 1.3, 2)
		
		for x,y,w,h in faces:
			roi = gray[y:y+h, x:x+w]
			roi = cv2.resize(roi, dsize=(160, 160))
			
			label = recog.predict(roi)

			if label[1] < 100:
				print(label)
				cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
				cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

		cv2.imshow("identify", frm)

		if cv2.waitKey(1) == 27:
			cv2.destroyAllWindows()
			cap.release()
			break

df = pd.read_excel('names_ids.xlsx')
names = list(df['Name'])
ids = list(df['ID'])

labelslist = {}
for name, id in zip(names, ids):
	id = str(id)
	labelslist[id] = name
print(labelslist)

recog = cv2.face.LBPHFaceRecognizer_create()

recog.read('model600.yml')

real_time_recognition_test()

{'1': 'Sunitha mam', '14': 'Ashish', '20': 'Chetan', '33': 'Kushal', '34': 'Lakshmi', '49': 'Muzammil', '54': 'NIthish', '59': 'Pavankumar', '78': 'Sanjana', '80': 'Shamant', '83': 'Shivani', '89': 'Suhas', '405': 'Umar'}
(59, 99.19635065854894)
(14, 94.75942307639477)
(54, 93.07938291674654)
(14, 98.33169946267165)
(20, 97.48119937649622)
(20, 92.21929441103906)
(20, 84.38384694389994)
(20, 83.96004469190842)
(20, 85.38211376994144)
(14, 94.48913609751651)
(20, 83.1632900382791)
(20, 81.12847487382147)
(20, 83.2862165996681)
(20, 82.40680837488759)
(20, 84.58031400897204)
(14, 91.09706056066152)
(20, 83.77499143645439)
(49, 82.9020350310035)
(20, 81.74270480098987)
(20, 80.51971809040393)
(20, 83.7259891704627)
(20, 82.95694437568908)
(33, 82.72090880106965)
(20, 83.58830701362218)
(20, 85.97869132390433)
(49, 93.6902520622986)
(20, 97.50338770933041)
(49, 90.56207281635395)
(49, 88.18383518045562)
(20, 94.24013226765234)
(20, 82.97256954419963)
(20, 84.8183329008661)
(49, 90.28808589

# viewing the registered students

In [6]:
from tkinter import ttk

def display_student_list():
    # Read student names and IDs from an Excel file
    df = pd.read_excel('names_ids.xlsx')

    root = tk.Tk()
    root.title("Student List")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = tk.Frame(canvas, width=400, height=300)  # Set minimum size for the frame

    # Configure the canvas and scrollbar
    def update_scrollregion(*args):  # Accept any arguments passed by events
        canvas.configure(scrollregion=canvas.bbox("all"))

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)
    scrollable_frame.bind("<Configure>", update_scrollregion)

    # Add heading labels for IDs and Names
    id_label = tk.Label(scrollable_frame, text="ID", font=("Arial", 12, "bold"))
    name_label = tk.Label(scrollable_frame, text="Name", font=("Arial", 12, "bold"))
    id_label.grid(row=0, column=0, sticky="w", padx=10)
    name_label.grid(row=0, column=1, sticky="w", padx=10)

    for index, row in df.iterrows():
        id_label = tk.Label(scrollable_frame, text=f"{row['ID']}")
        name_label = tk.Label(scrollable_frame, text=f"{row['Name']}")

        id_label.grid(row=index+1, column=0, sticky="w", padx=10)
        name_label.grid(row=index+1, column=1, sticky="w", padx=10)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    root.bind("<exclam>", lambda event: root.destroy())

    root.mainloop()

# viewing or downloading attendance file

In [7]:
from tkinter import ttk

def display_attendance():
    # Read student names and IDs from an Excel file
    df =pd.read_excel("students.xlsx")
    date_today = datetime.now().strftime("%Y-%m-%d")
    df = df[["Name","ID",date_today]]

    root = tk.Tk()
    root.title("Attendance List")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = tk.Frame(canvas, width=400, height=300)  # Set minimum size for the frame

    # Configure the canvas and scrollbar
    def update_scrollregion(*args):  # Accept any arguments passed by events
        canvas.configure(scrollregion=canvas.bbox("all"))

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)
    scrollable_frame.bind("<Configure>", update_scrollregion)

    # Add heading labels for IDs and Names
    id_label = tk.Label(scrollable_frame, text="ID", font=("Arial", 12, "bold"))
    name_label = tk.Label(scrollable_frame, text="Name", font=("Arial", 12, "bold"))
    time_label = tk.Label(scrollable_frame, text="Time", font=("Arial", 12, "bold"))
    id_label.grid(row=0, column=0, sticky="w", padx=10)
    name_label.grid(row=0, column=1, sticky="w", padx=10)
    time_label.grid(row=0, column=2, sticky="w", padx=10)

    for index, row in df.iterrows():
        id_label = tk.Label(scrollable_frame, text=f"{row['ID']}")
        name_label = tk.Label(scrollable_frame, text=f"{row['Name']}")
        time_entry = tk.Label(scrollable_frame, text=f"{row[date_today]}")
        id_label.grid(row=index+1, column=0, sticky="w", padx=10)
        name_label.grid(row=index+1, column=1, sticky="w", padx=10)
        time_entry.grid(row=index+1, column=2, sticky="w", padx=10)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    root.bind("<exclam>", lambda event: root.destroy())

    root.mainloop()

# Main or GUI

In [1]:
window = tk.Tk()
window.title("AUTOMATED ATTENDANCE MANAGEMENT SYSTEM")

window.geometry('1080x720')
window.config(bg='#607B9E')


# Create a frame for the buttons
button_frame = tk.Frame(window, borderwidth=1, relief="groove", bg="#333333")
button_frame.grid(row=2, column=0, padx=100, pady=50)

# Set font for the label
label_font = font.Font(size=25, weight='bold', family='Helvetica')

label_title = tk.Label(window, text="ATME College of Engineering", bg="#e8de1a", font=label_font)
label_title.grid(row=0, column=0, columnspan=1, padx=(20, 20), pady=(60, 5)), 

# Create and position the label
label_title = tk.Label(window, text="Automated Attendance Management System", bg="#e8de1a", font=label_font)
label_title.grid(row=1, column=0, columnspan=1, padx=(150,150), pady=(40, 40))

# Create and position the buttons within the frame
register = tk.Button(button_frame, text="REGISTER", width=13, height=2, bg="#e8de1a", command=register, font=label_font)
register.grid(row=1, column=0, padx=8, pady=8)

attendance = tk.Button(button_frame, text="ATTENDANCE", width=13, height=2, bg="#e8de1a", command=mark_attendance4, font=label_font)
attendance.grid(row=1, column=1, padx=8, pady=8)

students = tk.Button(button_frame, text="STUDENTS", width=13, height=2, bg="#e8de1a", command=display_student_list, font=label_font)
students.grid(row=2, column=0, padx=8, pady=8)

report = tk.Button(button_frame, text="REPORT", width=13, height=2, bg="#e8de1a", command=display_attendance, font=label_font)
report.grid(row=2, column=1, padx=8, pady=8)

# Center the button frame horizontally using `sticky`
# button_frame.grid(columnspan=1,padx=(60,60), pady=(120,120) ,sticky="w")  # Adjust here


window.bind("<Escape>", lambda event: window.destroy())

window.mainloop()

NameError: name 'tk' is not defined